In [ ]:
#Test our best mamba model and transformer solver on 10k test set
import torch
from model import seq2seq_generate_tour,MambaFull, compute_tour_length,generate_data
from benchmarks.transformer_model import TSP_net
from tqdm import tqdm
#from benchmarks.benchmark_solvers import greedy_tsp

coord_dim = 2
city_count = [50,100]
test_size=10000
device = 'cuda'

class DotDict(dict):
    def __init__(self, **kwds):
        self.update(kwds)
        self.__dict__ = self


checkpoints = [torch.load('mamba/checkpoints/big/share_50_big_fix_2.pt'),
               torch.load('mamba/checkpoints/big/BIG_city100_2.pt')]


for i, checkpoint_m in enumerate(checkpoints):
    if i>0:
        break
    args = checkpoint_m['args']
    model = MambaFull(args.d_model, args.city_count, args.nb_layers, args.coord_dim, args.mlp_cls,args.B, args.reverse,args.reverse_start,args.mamba2,args.last_layer).to(device)
    model.load_state_dict(checkpoint_m['model_baseline_state_dict'])
    model.eval()
    
    inputs = generate_data(device, 10, city_count, coord_dim=2 , start = 2)


    Loss_mamba = []
    with torch.no_grad():
        for j in tqdm(range(500)):
            tours ,Logprobofactions = seq2seq_generate_tour(device,model,inputs,deterministic=True,lastlayer=args.last_layer,sum_logactions=False)
            Loss_mamba.append(compute_tour_length(inputs,tours))
